In [4]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import load_model

### Sprawdzic SSIM bez sieci dla przykładowych zdjęć
https://ourcodeworld.com/articles/read/991/how-to-calculate-the-structural-similarity-index-ssim-between-two-images-with-python

In [98]:
from tqdm import tqdm
test_path = "../data/chest_xray/test"
dsize =(64, 64)

labels = {"NORMAL":0, "PNEUMONIA":1}

def load_images(dir_path, labels, dsize = (32, 32)):
    """loading images"""
    X_image, y_image = [], []
    for label_name, Id in labels.items():
        for jpeg in tqdm(os.listdir(os.path.join(dir_path, label_name))):
            path = os.path.join(dir_path, label_name, jpeg)
            image = load_img(path, target_size=dsize)
            X_image.append(image)
            #self.X_image.append(img_to_array(image)/255.0)
            y_image.append(Id)
    return X_image, y_image

X_test_org, y_test = load_images(test_path, labels, dsize = dsize)

100%|██████████| 234/234 [00:01<00:00, 147.40it/s]
0it [00:00, ?it/s]


In [100]:
import random
def preprocess_images(images, y, normalize = True):
    X = []
    for i in tqdm(range(len(images))):
        img = img_to_array(images[i]).astype('float32')
        if normalize:
            img = img/255.0
        X.append(img)
    #Shuffle data
    dataset = list(zip(X, y))
    random.shuffle(dataset)

    X, y = zip(*dataset)
    return np.array(X), np.array(y)

normalize = True
x_test, y_test =preprocess_images(X_test_org, y_test, normalize=normalize)
print("Shape of X_test:", x_test.shape)

100%|██████████| 234/234 [00:00<00:00, 28005.91it/s]

Shape of X_test: (234, 64, 64, 3)


In [2]:
def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred,1.0))

In [121]:
model = load_model('/home/iotg/work/SWA/SWA/models/pneumonia_ae.h5', custom_objects={'SSIMLoss': SSIMLoss})

In [9]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 64, 64, 3)]       0         
                                                                 
 conv_1 (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 batchnorm_1 (BatchNormaliza  (None, 64, 64, 32)       128       
 tion)                                                           
                                                                 
 leaky_relu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
                                                                 
 conv_2 (Conv2D)             (None, 32, 32, 64)        18496     
                                                                 
 batchnorm_2 (BatchNormaliza  (None, 32, 32, 64)       256       
 tion)                                                     

In [38]:
import gradio as gr

In [131]:
def classify_image(inp):
    inp = inp.resize((64, 64))
    img_array = img_to_array(inp).astype('float32')
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array/255.0
    model = load_model('/home/iotg/work/SWA/SWA/models/pneumonia_ae.h5', custom_objects={'SSIMLoss': SSIMLoss})
    prediction = model.predict(img_array)
    output = SSIMLoss(prediction, x_test[1])
    #prediction = np.squeeze(prediction, axis=0)
    #return prediction
    return np.max(inp), np.max(img_array), img_array.shape, output

In [144]:
gr.close_all()


Closing server running on port: 7861
Closing server running on port: 7860


In [145]:
gr.Interface(fn=classify_image, 
             inputs=gr.Image(type="pil"),
             outputs="text",
             examples=["/home/iotg/work/SWA/SWA/data/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg"]).launch(server_port=7861 , share=True, debug=True)

Running on local URL:  http://127.0.0.1:7861/
1/1 [==============================] - 0s 126ms/step


KeyboardInterrupt: 